<a href="https://colab.research.google.com/github/buwuyou/CTCN_2020_EE/blob/master/CTCN_time_series_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This tutorial will establish a foundation for time series analysis of remote sensing data (Sentinel-2 time series data), including:

1. Calculte a popular vegetation indice

2. Apply a simple linear analysis on the 4-year time series

3. Plot the time series for known locations

4. Export the trend map for your study area


Prerequisites: CTCN 101

### Install, Import, Authenticate and Initialize EE Python API

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [0]:
!pip install -q earthengine-api

# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()

In [0]:
from pprint import pprint 

import folium
from folium import plugins
print(folium.__version__)
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# NDVI 101

The normalised difference vegeation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing assessed greenness of vegetation.

>$NDVI=(NIR-R)/(NIR+R)$

In [0]:
def addNDVI(img):
    """A function to compute Sentinel-2 NDVI."""
    ndvi = img.expression('float(b("B8") - b("B4")) / (b("B8") + b("B4"))').rename('ndvi')   
    return img.addBands([ndvi])

In [0]:
def ESAcloudMask(img):
    ''' Sentinel-2 Bits 10 & 11 are clouds & cirrus, so set to 0. '''
    qa = img.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return img.updateMask(clear) 

In [0]:
def addTime(image):
  '''This function adds a band representing the image timestamp.'''
  # Convert milliseconds from epoch to years to aid in interpretation of the following trend calculation.
  return image.addBands(image.metadata('system:time_start').divide(1000 * 60 * 60 * 24 * 365))


In [0]:
def s2col_monthly(feat):
      '''A function to create s2 monthly composites for three years'''
      bandNamesAll = ['ndvi']
      monthDifference = ee.Date('2019-01-01').advance(1, 'month').millis().subtract(ee.Date('2019-01-01').millis());
      listMap = ee.List.sequence(ee.Date('2016-01-01').millis(), ee.Date('2020-01-01').millis(), monthDifference);

      s2monthIC = (ee.ImageCollection.fromImages(listMap.map(lambda dateMillis: (
         (ee.ImageCollection('COPERNICUS/S2')
                            .filterBounds(feat)
                            .filter(ee.Filter.And(
                              ee.Filter.date(ee.Date(dateMillis), ee.Date(dateMillis).advance(1, 'month')),
                              ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)))
                            .map(ESAcloudMask)
                            
                            )
         .median()
         .clip(feat)
         .set('system:time_start',dateMillis)
         .set('year', ee.Date(dateMillis).get('year').toInt())
         .set('month', ee.Date(dateMillis).get('month').toInt())
         )
      )))
      
      return s2monthIC.map(addNDVI).select(bandNamesAll).map(addTime)

### Define AOI

In [0]:
# Define an AOI
AOI = ee.Geometry.Polygon(
        [[[
              89.40673828125,
              21.37124437061831
            ],
            [
              90.90087890624999,
              21.37124437061831
            ],
            [
              90.90087890624999,
              22.471954507739227
            ],
            [
              89.40673828125,
              22.471954507739227
            ],
            [
              89.40673828125,
              21.37124437061831
            ]]])

### Create Sentinel-2 monthly observation

In [0]:
# Fetch a Sentinel-2 monthly images.
imgs = s2col_monthly(AOI)

In [0]:
pprint ({'Sentinel-2 Monthly Images since Jan, 2016': imgs.size().getInfo()})
pprint ({'Sentinel-2 Monthly Image': imgs.first().getInfo()})

# Simple Linear Trend Analysis

In statistics, linear regression is a linear approach to modeling the relationship between a scalar response (e.g NDVI) and one or more explanatory variables (e.g. time). The case of one explanatory variable is called simple linear regression. 

Consider the following linear model, where **e** is a random error
> $ NDVI_t = β_{0} + β_{1}t + e^{t}$

It presents a time series model that a scalar response of NDVI is explained by the time (t). When the time steps increases, a time series of NDVI is such composited and indexed in time order. $β_{0}$ is socalled as offset, and $β_{1}$ is slope, which indicates the direction and altitude of changes during time, socalled as trends.



In [0]:
#  Select the bands to model with the independent variable first.
trend = imgs.select(['system:time_start', 'ndvi']).reduce(ee.Reducer.linearFit())# Compute the linear trend over time.
pprint ({'Sentinel-2 Trend Image': trend.getInfo()})

### Display a trend map

In [0]:
orange2greenPalette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
mapid = trend.getMapId({'bands': ['scale'], 'min': -0.1, 'max': 0.1, 'palette': orange2greenPalette})
map = folium.Map(location=[22., 90.], zoom_start=10)
# Add custom basemaps
basemaps['Google Maps'].add_to(map)
basemaps['Google Satellite Hybrid'].add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Sentinel-2 Trend Map 2016-2019',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Display the first year NDVI and last year NDVI together with a trend map

Change the monthID, then the according monthly NDVI image will be displayed below for the year of 2016 and 2019, the greener the image is, the more greeness the vegetation cannopy being reflected


In [0]:
monthID = 2
yearIDstart = 2016
yearIDend = 2019


imageS = imgs.select('ndvi').filter(ee.Filter.eq('year',yearIDstart)).filter(ee.Filter.eq('month',monthID)).first()
imageE = imgs.select('ndvi').filter(ee.Filter.eq('year',yearIDend)).filter(ee.Filter.eq('month',monthID)).first()

mapid = imageS.getMapId({'min': 0, 'max': 0.8, 'palette': orange2greenPalette})
mapid1 = imageE.getMapId({'min': 0, 'max': 0.8, 'palette': orange2greenPalette})
mapid2 = trend.getMapId({'bands': ['scale'], 'min': -0.1, 'max': 0.1, 'palette': orange2greenPalette})
map = folium.Map(location=[22., 90.], zoom_start=8)
folium.TileLayer(
    tiles=mapid1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Sentinel NDVI for year {yearIDstart} and month {monthID}',
  ).add_to(map)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Sentinel NDVI for year {yearIDend} and month {monthID}',
  ).add_to(map)

folium.TileLayer(
    tiles=mapid2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Sentinel-2 Trend Map 2016-2019',
  ).add_to(map)
map.add_child(folium.LayerControl())
folium.LatLngPopup().add_to(map)
map

# Plot time series for different land use/cover classes

Please use the NDVI and trend map displayed above, to record few coordinates one has interests to understand the inter-annual and intra-annual development of NDVI for three years, such as

>Negative trend: 90.3102, 22.0266

>Mangrove: 89.8031, 22.1562

>Agriculture: 89.9708, 22.1110

>Water: 89.8970, 22.1199

In [0]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt
from pprint import pprint
import datetime
import pandas as pd
%matplotlib inline

### To fetch time series data from EE

In [0]:
def GetDataFrame(coords):
    '''a function to get the ndvi time series for a point geometry'''
    pnt = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = imgs.select('ndvi').getRegion(geometry=pnt, scale=10)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['datetime', 'ndvi']]
    return data


### Plot a mangrove pixel

In [0]:
df = GetDataFrame([89.8031, 22.1562])
x_time = np.asarray(df['datetime'])
y_ndvi = np.asarray(df['ndvi'])

In [0]:
fig = plt.figure(figsize=(15,4))
plt.plot(x_time, y_ndvi)

### Plot a pixel experiecing decreasing trend

In [0]:
df = GetDataFrame([90.3102, 22.0266])
x_time = np.asarray(df['datetime'])
y_ndvi = np.asarray(df['ndvi'])

In [0]:
fig = plt.figure(figsize=(15,4))
plt.plot(x_time, y_ndvi)


# Exercises

### 1. Check the time series of some locations 
Record few locations' coordinates on your own interests, and plot the NDVI time series of them

### 2. Export a trend map for your own AOI

Use the code provided above, change the AOI to cover your own study area, run the codes to compute the trend and export
Share us what do you find in the trend map?

In [0]:
# define a smaller aoi for exporting
export_aoi = ee.Geometry.Polygon([
          [
            [
              89.57324981689453,
              22.354203655362834
            ],
            [
              89.70096588134764,
              22.354203655362834
            ],
            [
              89.70096588134764,
              22.4300707662349
            ],
            [
              89.57324981689453,
              22.4300707662349
            ],
            [
              89.57324981689453,
              22.354203655362834
            ]
          ]
        ])
task= ee.batch.Export.image.toDrive(
            image = trend.select('scale'),
            description = f'CTCN101_Trend_S2_2016_2019',
            folder = f'CTCN101',
            region= export_aoi.getInfo()["coordinates"],    
            scale= 10,
            maxPixels= int(2e9),
            crs='EPSG:4326')
task.start()

# Block until the task completes.
print('Running image export to Google Drive...')
import time
while task.active():
  time.sleep(30)

# Error condition
if task.status()['state'] != 'COMPLETED':
  print('Error with image export.')
else:
  print('Image export completed.')